# 일단 파일에서 test/train 데이터 분리

In [2]:
import os, glob, random, shutil
from sklearn.model_selection import train_test_split

# === 경로 설정 ===
DATA_ROOT = "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725"
IMG_DIR   = os.path.join(DATA_ROOT, "images")
LAB_DIR   = os.path.join(DATA_ROOT, "labels")
OUT_DIR   = os.path.join(DATA_ROOT, "yolo_dataset")

# reproducibility
random.seed(42)

# === 이미지 리스트 ===
images = sorted(glob.glob(os.path.join(IMG_DIR, "*.jpg")))
print("총 이미지 수:", len(images))

# === train/val 분리 (8:2) ===
train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

def get_label_path(img_path):
    name = os.path.basename(img_path).replace(".jpg", ".txt")
    return os.path.join(LAB_DIR, name)

# === 출력 폴더 생성 ===
for d in ["images/train", "images/val", "labels/train", "labels/val"]:
    os.makedirs(os.path.join(OUT_DIR, d), exist_ok=True)

def copy_pairs(img_list, split):
    for img in img_list:
        lab = get_label_path(img)
        if not os.path.exists(lab):
            print("[WARN] 라벨 없음:", img)
            continue
        shutil.copy2(img, os.path.join(OUT_DIR, "images", split, os.path.basename(img)))
        shutil.copy2(lab, os.path.join(OUT_DIR, "labels", split, os.path.basename(lab)))

copy_pairs(train_imgs, "train")
copy_pairs(val_imgs, "val")

print("train 개수:", len(train_imgs))
print("val 개수:", len(val_imgs))



총 이미지 수: 2080
train 개수: 1664
val 개수: 416


# 라벨 갯수 확인

In [1]:
import os, glob
from collections import Counter

# 경로 설정 (네 환경에 맞춤)
DATA_ROOT = "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725/yolo_dataset"
LABEL_ROOT = os.path.join(DATA_ROOT, "labels")
IMG_ROOT   = os.path.join(DATA_ROOT, "images")

# 모든 라벨 파일 수집
label_files = sorted(glob.glob(os.path.join(LABEL_ROOT, "**/*.txt"), recursive=True))
print("라벨 파일 개수:", len(label_files))

cls_counter = Counter()
bad_lines = []
empty_labels = []
for lf in label_files:
    has_line = False
    with open(lf, "r") as f:
        for ln, line in enumerate(f, start=1):
            line = line.strip()
            if not line:
                continue
            has_line = True
            toks = line.split()
            try:
                cid = int(toks[0])
                # YOLO: class x y w h (0~1)
                x, y, w, h = map(float, toks[1:5])
                # 간단 유효성 체크
                if not (0 <= x <= 1 and 0 <= y <= 1 and 0 < w <= 1 and 0 < h <= 1):
                    bad_lines.append((lf, ln, line))
                cls_counter[cid] += 1
            except Exception as e:
                bad_lines.append((lf, ln, line))
    if not has_line:
        empty_labels.append(lf)

print("클래스 분포:", cls_counter)
if empty_labels:
    print("[WARN] 박스가 하나도 없는 라벨 파일 수:", len(empty_labels))

if bad_lines:
    print("[WARN] 비정상 라벨 라인(앞 5개):")
    for t in bad_lines[:5]:
        print(" ", t)

라벨 파일 개수: 2080
클래스 분포: Counter({0: 1528, 1: 1389})
[WARN] 박스가 하나도 없는 라벨 파일 수: 752


# 쓸모없는 라벨 삭제

In [4]:
import os, glob

# yolo_dataset 경로 (train/val 포함해서 전체 탐색)
LABEL_ROOT = "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725/yolo_dataset/labels"

remove_classes = {2, 3, 4, 5}

label_files = sorted(glob.glob(os.path.join(LABEL_ROOT, "**/*.txt"), recursive=True))
print("라벨 파일 개수:", len(label_files))

changed, emptied = 0, 0
for lf in label_files:
    with open(lf, "r") as f:
        lines = f.readlines()
    new_lines = []
    for line in lines:
        toks = line.strip().split()
        if not toks:
            continue
        cid = int(toks[0])
        if cid in remove_classes:
            continue  # 버림
        new_lines.append(line)
    if new_lines != lines:
        changed += 1
        with open(lf, "w") as f:
            f.writelines(new_lines)
    if not new_lines:
        emptied += 1

print(f"수정된 라벨 파일: {changed}개")
print(f"완전히 비어버린 라벨 파일: {emptied}개")


라벨 파일 개수: 2080
수정된 라벨 파일: 91개
완전히 비어버린 라벨 파일: 752개


In [9]:
from ultralytics import YOLO

DATA_YAML = "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725/data.yaml"

model = YOLO("yolov8s.pt")  # 가볍게 시작, OOM 나면 yolov8n.pt

results = model.train(
    data=DATA_YAML,
    epochs=100,          # 필요시 50~300 조절
    imgsz=640,           # 512~1024 사이 조절
    batch=16,            # OOM 나면 8/4로 낮추기
    device=0,            # GPU 0번, CPU는 "cpu"
    workers=8,           # dataloader worker
    project="/home/dw/ws_job_msislab/Golf_Project/runs_yolo",
    name="golf_yv8s",
    verbose=True
)

Ultralytics 8.3.175 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=golf_yv8s2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patienc

AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 5342.6±3253.7 MB/s, size: 3257.2 KB)


train: Scanning /home/dw/ws_job_msislab/Golf_Project/data/Data_20250725/yolo_dataset/labels/train... 1664 images, 604 backgrounds, 0 corrupt: 100%|██████████| 1664/1664 [00:00<00:00, 4234.20it/s]

train: New cache created: /home/dw/ws_job_msislab/Golf_Project/data/Data_20250725/yolo_dataset/labels/train.cache


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4869.9±2606.4 MB/s, size: 3331.9 KB)


...

    100/100      3.57G      1.152     0.6632      1.035         23        640: 100%|██████████| 104/104 [00:16<00:00,  6.31it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:01<00:00,  8.30it/s]

                   all        416        600      0.653      0.604      0.652      0.353



100 epochs completed in 0.516 hours.
Optimizer stripped from /home/dw/ws_job_msislab/Golf_Project/runs_yolo/golf_yv8s2/weights/last.pt, 22.5MB
Optimizer stripped from /home/dw/ws_job_msislab/Golf_Project/runs_yolo/golf_yv8s2/weights/best.pt, 22.5MB

Validating /home/dw/ws_job_msislab/Golf_Project/runs_yolo/golf_yv8s2/weights/best.pt...
Ultralytics 8.3.175 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
Model summary (fused): 72 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:01<00:00,  7.50it/s]


                   all        416        600        0.6      0.674      0.676      0.356
                 divot        189        311       0.54      0.556      0.562      0.287
           fixed_divot        171        289       0.66      0.792       0.79      0.425
Speed: 0.2ms preprocess, 2.2ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /home/dw/ws_job_msislab/Golf_Project/runs_yolo/golf_yv8s2


Copied train=1664, val=416
to augment: 832
added (per transform): {'blur': 832, 'jpeg': 832, 'downup': 832}
Done → /home/dw/ws_job_msislab/Golf_Project/data/Data_20250725/yolo_dataset2
